In [1]:
import pandas as pd

In [2]:
from stop_words import get_stop_words

In [3]:
stop_words = get_stop_words('english')

In [11]:
stop_words = stop_words + ['patient','inclusion','criteria','aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn']

In [12]:
print(stop_words)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

In [13]:
data = pd.read_csv('inclusion_criteria.csv',sep='|')

In [14]:
data.head()

,Unnamed: 0,inclusion,nct_id
0,0,Inclusion Criteria for Retention Phase: Indivi...,NCT04464980
1,0,1. 18 years of age or older;,NCT04464980
2,0,2. Meet DSM-5 criteria for current opioid use ...,NCT04464980
3,0,3. Seeking treatment for opioid use disorder a...,NCT04464980
4,0,"4. If choosing buprenorphine, willing to be ra...",NCT04464980


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
cv = CountVectorizer(max_df=0.80, min_df=2, stop_words=(stop_words),ngram_range=(1,3) )

In [17]:
dtm = cv.fit_transform(data['inclusion'])

In [18]:
dtm

<17203x51428 sparse matrix of type '<class 'numpy.int64'>'
	with 422860 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.decomposition import LatentDirichletAllocation

In [20]:
LDA = LatentDirichletAllocation(n_components=40,random_state=42)

In [21]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(n_components=40, random_state=42)

In [22]:
len(cv.get_feature_names())

51428

In [23]:
import random

In [24]:
for i in range(10):
    random_word_id = random.randint(0,599)
    print(cv.get_feature_names()[random_word_id])

130 diastolic bp
10 meters
10 worst
115
10 mg dl
10º coronal
104
12 hours last
100 mmhg
10 platelet count


In [25]:
len(LDA.components_)

40

In [26]:
LDA.components_

array([[0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       ...,
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 2.05806579, ..., 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 2.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ]])

In [27]:
len(LDA.components_[0])

51428

In [28]:
single_topic = LDA.components_[0]

In [29]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([    0, 33124, 33125, ...,  4417, 21798, 33674], dtype=int64)

In [30]:
# Word least representative of this topic
single_topic[322]

0.025

In [31]:
# Word most representative of this topic
single_topic[255]

0.025

In [32]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([  727, 25389, 48120, 50942, 33725,  4432,  8576,  4417, 21798,
       33674], dtype=int64)

In [33]:
top_word_indices = single_topic.argsort()[-10:]

In [34]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

18
least
unit
years
patients aged
aged 18
care
aged
hospital
patients


In [35]:
def getTopicKeyWorkds():
    keywords = []
    for index,topic in enumerate(LDA.components_):
        keywords.append([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
#         print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
#         print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
#         print('\n')
    return keywords
    

In [36]:
topicKeywords = getTopicKeyWorkds() 

In [37]:
topicData = pd.DataFrame()

In [38]:
#df['newcolumn'], df['newcolumn2'] = zip(tuple)

topicData['topic'] = topicKeywords

In [39]:
for index, data_ in enumerate(topicData['topic']):
    print (index)
    print (data_)

0
['university', 'aged 18 years', 'disease', 'patients aged 18', 'intensive', '18', 'least', 'unit', 'years', 'patients aged', 'aged 18', 'care', 'aged', 'hospital', 'patients']
1
['history', 'score', 'disease', 'hormonal contraception', 'subjects', 'clinical', 'bilateral', 'oral', 'following', 'treatment', 'knee', 'months', 'least', 'pain', 'patients']
2
['reaction rt', 'reaction rt pcr', 'without', 'minute', 'treatment', 'pressure', 'reverse', 'per', 'polymerase chain', 'polymerase chain reaction', 'normal', 'chain reaction', 'polymerase', 'reaction', 'chain']
3
['results', 'screening', 'significant', 'laboratory', 'physical examination', 'clinically', 'health', 'good', 'medical history', 'investigator', 'clinical', 'physical', 'history', 'examination', 'medical']
4
['subjects must', 'received', 'one following', 'measurable lesion', 'prior', 'following', 'treatment', 'subjects', 'chemotherapy', 'measurable', 'lesion', 'least one', 'must', 'one', 'least']
5
['willing able comply', 'ab

In [40]:
dtm

<17203x51428 sparse matrix of type '<class 'numpy.int64'>'
	with 422860 stored elements in Compressed Sparse Row format>

In [41]:
dtm.shape 

(17203, 51428)

In [42]:
len(data)

17203

In [43]:
topic_results = LDA.transform(dtm)

In [44]:
topic_results.shape

(17203, 40)

In [45]:
topic_results[0]

array([0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 ,
       0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 ,
       0.0015625 , 0.0015625 , 0.29183379, 0.0015625 , 0.0015625 ,
       0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 ,
       0.0015625 , 0.08796231, 0.14199869, 0.0015625 , 0.0015625 ,
       0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 ,
       0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 , 0.0015625 ,
       0.0015625 , 0.42195521, 0.0015625 , 0.0015625 , 0.0015625 ])

In [46]:
topic_results[0].round(2)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.29, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.09,
       0.14, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.42, 0.  , 0.  , 0.  ])

In [47]:
topic_results[0].argmax()

36

In [48]:
topic_results.argmax(axis=1)

array([36, 27, 16, ..., 32, 32, 32], dtype=int64)

In [49]:
data.shape

(17203, 3)

In [50]:
data['Topic'] = topic_results.argmax(axis=1)

In [51]:
data.head(10)

,Unnamed: 0,inclusion,nct_id,Topic
0,0,Inclusion Criteria for Retention Phase: Indivi...,NCT04464980,36
1,0,1. 18 years of age or older;,NCT04464980,27
2,0,2. Meet DSM-5 criteria for current opioid use ...,NCT04464980,16
3,0,3. Seeking treatment for opioid use disorder a...,NCT04464980,1
4,0,"4. If choosing buprenorphine, willing to be ra...",NCT04464980,29
5,0,5. Willing to be randomized to either MM (stan...,NCT04464980,10
6,0,6. In good-enough general health (meaning good...,NCT04464980,10
7,0,7. Willing and able to provide written informe...,NCT04464980,22
8,0,8. Able to speak English sufficiently to under...,NCT04464980,22
9,0,"9. If female of childbearing potential, willin...",NCT04464980,11


In [52]:
data.groupby(['Topic']).count()

,Unnamed: 0,inclusion,nct_id
Topic,,,
0,1464,1464,1464
1,351,351,351
2,208,208,208
3,354,354,354
4,320,320,320
5,378,378,378
6,293,293,293
7,271,271,271
8,612,612,612


In [53]:
sorted_data = data.sort_values(by=['Topic'])
sorted_data.to_csv('topic_sorted2.csv',sep = '|')

In [54]:
topicData.to_csv('topicKeywords2.csv',sep = '|')

In [55]:
#Age_related = data[data['Topic']==7]

In [56]:
#Age_related.to_csv('Age_related.csv',sep = '|')